In [1]:
import pandas as pd
import numpy as np

In [2]:
test = pd.read_csv('test.csv')

In [3]:
train = pd.read_csv('train.csv')

In [4]:
#vedo per ogni colonna quali sono i valori unici
#non ci sono valori nan
#noto che 00.00 è l'ora per ogni AppointmentDay
for el in train.columns:
    print("Colonna:"+el+"",train[el].unique())

Colonna:No-show ['No' 'Yes']
Colonna:PatientId [1.16723945e+11 2.37549850e+13 9.43571216e+10 ... 5.53613414e+14
 8.87375234e+14 1.24658919e+13]
Colonna:AppointmentID [5778236 5739813 5683899 ... 5622058 5717386 5723664]
Colonna:Gender ['M' 'F']
Colonna:ScheduledDay ['2016-06-06T15:14:21Z' '2016-05-25T14:10:29Z' '2016-05-11T08:00:20Z' ...
 '2016-05-13T15:43:41Z' '2016-04-26T11:16:09Z' '2016-05-20T08:29:12Z']
Colonna:AppointmentDay ['2016-06-08T00:00:00Z' '2016-05-30T00:00:00Z' '2016-05-11T00:00:00Z'
 '2016-05-16T00:00:00Z' '2016-05-17T00:00:00Z' '2016-05-06T00:00:00Z'
 '2016-06-03T00:00:00Z' '2016-05-12T00:00:00Z' '2016-05-25T00:00:00Z'
 '2016-05-02T00:00:00Z' '2016-05-09T00:00:00Z' '2016-05-31T00:00:00Z'
 '2016-05-18T00:00:00Z' '2016-05-20T00:00:00Z' '2016-06-02T00:00:00Z'
 '2016-06-01T00:00:00Z' '2016-05-05T00:00:00Z' '2016-05-13T00:00:00Z'
 '2016-05-04T00:00:00Z' '2016-05-03T00:00:00Z' '2016-06-07T00:00:00Z'
 '2016-05-19T00:00:00Z' '2016-05-10T00:00:00Z' '2016-04-29T00:00:00Z'
 '2016

In [5]:
#non ci sono valori nulli
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22106 entries, 0 to 22105
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   No-show         22106 non-null  object 
 1   PatientId       22106 non-null  float64
 2   AppointmentID   22106 non-null  int64  
 3   Gender          22106 non-null  object 
 4   ScheduledDay    22106 non-null  object 
 5   AppointmentDay  22106 non-null  object 
 6   Age             22106 non-null  int64  
 7   Neighbourhood   22106 non-null  object 
 8   Scholarship     22106 non-null  int64  
 9   Hipertension    22106 non-null  int64  
 10  Diabetes        22106 non-null  int64  
 11  Alcoholism      22106 non-null  int64  
 12  Handcap         22106 non-null  int64  
 13  SMS_received    22106 non-null  int64  
dtypes: float64(1), int64(8), object(5)
memory usage: 2.4+ MB


In [6]:
#dal giorno in cui è stato fissato l'appuntamento e il giorno effettivo della visita calcolo daysToAppointment
#daysToAppointment indica i giorni di attesa 
#trasformo le date da stringhe in datetime e poi faccio la differenza
#a questo punto le colonne ScheduledDay e AppointmentDay non serviranno più all'analisi
train.ScheduledDay = train.ScheduledDay.apply(np.datetime64)
train.AppointmentDay = train.AppointmentDay.apply(np.datetime64)
test.ScheduledDay = test.ScheduledDay.apply(np.datetime64)
test.AppointmentDay = test.AppointmentDay.apply(np.datetime64)

daysToAppointment = train.AppointmentDay - train.ScheduledDay
daysToAppointment = daysToAppointment.apply(lambda x: x.total_seconds() / (3600 * 24))
train['daysToAppointment'] = daysToAppointment

daysToAppointment = test.AppointmentDay - test.ScheduledDay
daysToAppointment = daysToAppointment.apply(lambda x: x.total_seconds() / (3600 * 24))
test['daysToAppointment'] = daysToAppointment


train['daysToAppointment']= train['daysToAppointment'].astype('int64')
test['daysToAppointment']= test['daysToAppointment'].astype('int64')

In [7]:
train.columns

Index(['No-show', 'PatientId', 'AppointmentID', 'Gender', 'ScheduledDay',
       'AppointmentDay', 'Age', 'Neighbourhood', 'Scholarship', 'Hipertension',
       'Diabetes', 'Alcoholism', 'Handcap', 'SMS_received',
       'daysToAppointment'],
      dtype='object')

In [8]:
train.daysToAppointment

0         1
1         4
2         0
3         3
4         0
         ..
22101    15
22102     6
22103     7
22104     0
22105     0
Name: daysToAppointment, Length: 22106, dtype: int64

In [9]:
#rimuovo le righe con degli eventuali futuri valori nulli
#rimuovo le colonne non utili ai fini dell'analisi: AppointmentID, PatientId e Neighbourhood, 'AppointmentDay', 'ScheduledDay'
#separo il target dalle predizioni sia per il validation set sia per il training set
train.dropna(axis=0, subset=['No-show', 'PatientId', 'AppointmentID', 'Neighbourhood',
                             'AppointmentDay', 'ScheduledDay'], inplace=True)
y_train = train['No-show']
X_train = train.drop(['No-show', 'PatientId', 'AppointmentID',  'Neighbourhood',
                     'AppointmentDay', 'ScheduledDay'], axis=1).copy()

test.dropna(axis=0, subset=['No-show', 'PatientId', 'AppointmentID', 'Neighbourhood',
                           'AppointmentDay', 'ScheduledDay'], inplace=True)
y_valid = test['No-show']
X_valid = test.drop(['No-show', 'PatientId', 'AppointmentID', 'Neighbourhood',
                    'AppointmentDay', 'ScheduledDay'], axis=1).copy()

#seleziono le colonne categoriche
cat_cols = [cname for cname in X_train.columns if 
                    X_train[cname].dtype == "object"]

# Seleziono le colonne numeriche
num_cols = [cname for cname in X_train.columns if 
                X_train[cname].dtype in ['int64', 'float64']]

In [10]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.tree import DecisionTreeClassifier

In [11]:
#pipeline per combinare in sequenza le diverse operazioni sui dati
#i valori mancanti vengono sostituiti con la media grazie all'imputazione
#le variabili categoriche vengono trasfprmate con il OneHot-encoder, il quale crea nuove colonne che indicano 
#presenza (o l'assenza) per ogni valore possibile presente nei dati
num_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median', missing_values=np.nan)),
    ('scaler', StandardScaler())
])
cat_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent', missing_values=np.nan)),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
]) 

# preprocessamento dei dati a seconda del loro tipo
preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_transformer, num_cols),
        ('cat', cat_transformer, cat_cols),
    ])

# mettendo il criterio entropia il root node  partiziona i dati usando la feature che fornisce il valore
#informazionale più alto
model = DecisionTreeClassifier(criterion='entropy', random_state=0, max_depth=5, splitter='best')


In [12]:
#riduco la dimensionalità con la feature selection 
from sklearn.feature_selection import SelectKBest, f_classif

# pipeline
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                               ('skb', SelectKBest(f_classif, k = 10)),
                              ('model', model)
                             ])

# alleno il modello
my_pipeline.fit(X_train, y_train)

# predizioni
preds = my_pipeline.predict(X_valid)

In [13]:
from sklearn.metrics import accuracy_score
model=accuracy_score(preds, y_valid)
model


0.796335670662746